- [Youtube](https://www.youtube.com/watch?v=qMFKsMeE6fM&ab_channel=JeffHeaton)
- [GitHub](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_06_3_resnet.ipynb)

In [ ]:
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, AveragePooling2D, Input, Flatten, add
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
import numpy as np
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from keras.utils import to_categorical

## Resnet V1

In [ ]:
## load data
class_mapping = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabdefghnqrt'
train_df = pd.read_csv('../EMNIST_NEW/train.csv', header = None)
test_df = pd.read_csv('../EMNIST_NEW/test.csv', header = None)

x_train = np.array([np.transpose(train_df.values[idx,1:].reshape(28, 28), axes=[1,0]).astype('uint8')/255 for idx in range(1,len(train_df))])
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
y_train = to_categorical(train_df[0].values[1:])

x_test = np.array([np.transpose(test_df.values[idx,1:].reshape(28, 28), axes=[1,0]).astype('uint8')/255 for idx in range(1,len(test_df))])
x_test  = x_test.reshape(x_test .shape[0], x_test .shape[1], x_test.shape[2], 1)
y_test = to_categorical(test_df[0].values[1:])

print('Train')
print(x_train.shape, y_train.shape)
# print(pd.Series(y_train).value_counts())
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print('Test')
print(x_test.shape, y_test.shape)

# test_idx = 0
# test_img = x_train[test_idx]
# test_img = test_img.reshape(test_img.shape[0], test_img.shape[1], 1)
# plt.imshow(x_train[3], cmap='Greys_r')

# cv2.imshow('test', test_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
# import the necessary packages

#PYIMAGESEARCH
class ResNet:
    @staticmethod
    def residual_module(data, K, stride, chanDim, red=False,
        reg=0.0001, bnEps=2e-5, bnMom=0.9):
        # the shortcut branch of the ResNet module should be
        # initialize as the input (identity) data
        shortcut = data

        # the first block of the ResNet module are the 1x1 CONVs
        bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(data)
        
        act1 = Activation("relu")(bn1)
        conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False,
            kernel_regularizer=l2(reg))(act1)

        # the second block of the ResNet module are the 3x3 CONVs
        bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps,
            momentum=bnMom)(conv1)
        act2 = Activation("relu")(bn2)
        conv2 = Conv2D(int(K * 0.25), (3, 3), strides=stride,
            padding="same", use_bias=False,
            kernel_regularizer=l2(reg))(act2)

        # the third block of the ResNet module is another set of 1x1
        # CONVs
        bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps,
            momentum=bnMom)(conv2)
        act3 = Activation("relu")(bn3)
        conv3 = Conv2D(K, (1, 1), use_bias=False,
            kernel_regularizer=l2(reg))(act3)

        # if we are to reduce the spatial size, apply a CONV layer to
        # the shortcut
        if red:
            shortcut = Conv2D(K, (1, 1), strides=stride,
                use_bias=False, kernel_regularizer=l2(reg))(act1)

        # add together the shortcut and the final CONV
        x = add([conv3, shortcut])

        # return the addition as the output of the ResNet module
        return x

    @staticmethod
    def build(width, height, depth, classes, stages, filters,
        reg=0.0001, bnEps=2e-5, bnMom=0.9, dataset="cifar"):
        # initialize the input shape to be "channels last" and the
        # channels dimension itself
        inputShape = (height, width, depth)
        chanDim = -1

        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1

        # set the input and then apply a BN followed by CONV
        inputs = Input(shape=inputShape)
        x = BatchNormalization(axis=chanDim, epsilon=bnEps,
            momentum=bnMom)(inputs)
        x = Conv2D(filters[0], (3, 3), use_bias=False,
            padding="same", kernel_regularizer=l2(reg))(x)

        # loop over the number of stages
        for i in range(0, len(stages)):
            # initialize the stride, then apply a residual module
            # used to reduce the spatial size of the input volume
            stride = (1, 1) if i == 0 else (2, 2)
            x = ResNet.residual_module(x, filters[i + 1], stride,
                chanDim, red=True, bnEps=bnEps, bnMom=bnMom)

            # loop over the number of layers in the stage
            for j in range(0, stages[i] - 1):
                # apply a ResNet module
                x = ResNet.residual_module(x, filters[i + 1],
                    (1, 1), chanDim, bnEps=bnEps, bnMom=bnMom)

        # apply BN => ACT => POOL
        x = BatchNormalization(axis=chanDim, epsilon=bnEps,
            momentum=bnMom)(x)
        x = Activation("relu")(x)
        x = AveragePooling2D((8, 8))(x)
    
        # softmax classifier
        x = Flatten(input_shape = inputs)(x)
        x = Dense(classes, kernel_regularizer=l2(reg))(x)
        x = Activation("softmax")(x)

        # create the model
        model = Model(inputs, x, name="resnet")

        # return the constructed network architecture
        return model

In [ ]:
batch_size = 128 
epochs = 200 
augmentation = False 
num_classes = len(train_df[0].unique())
colors = 1 
input_shape = x_train.shape[1:]

subtract_pixel_mean = True
depth = colors * 6 + 2

print(input_shape)
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=5, min_delta = .00075)
model_checkpoint = ModelCheckpoint(f'ModelWeights/Mobilenet_Masks.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
# lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor = np.sqrt(.1), cooldown = 0, patience = 5, min_lr = .5e-6)
callbacks = [early_stopping, model_checkpoint, lr_reducer]


res_model = ResNet.build(28,28,1, num_classes, (3,3,3), (64,64,128,256), reg = .0005)


red_model.summary()

In [ ]:
assert False

In [ ]:


res_v1 = resnet_v1(input_shape = input_shape, depth = depth)

if subtract_pizel_mean == True: 
    x_train_mean = np.mean(x_train, axis = 0)
    x_train -= x_train_mean
    x_test -= x_train_mean

# if augmentation == True:
#     augmentation =ImageDataGenerator(rotation_range = 20, width_shift_range = .2, height_shift_range = .2, 
#                                                            horizontal_flip = True, shear_range = .15, 
#                                      fill_mode = 'nearest', zoom_range = .15)
#     augmentation.fit(x_train)
    
#     res_v1.fit_generator(augmentation.flow(x_train, y_train, batch_size = batch_size), epochs = epochs, 
#                          validation_data = (x_test, y_test), callbacks = callbacks)
# else: 
#     res_v1.fit(x_train, y_train, epochs = epochs, batch_size = batch_size, validation_data = (x_test, y_test), 
#                callbacks = callbacks, shuffle = True)



In [ ]:
assert False